In [1]:
import numpy as np
import pandas as pd
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

2025-12-15 02:37:49.030608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765766269.366686      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765766269.469193      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
df = pd.read_csv("/kaggle/input/data-csv-signe-language/dataset_keypoints.csv")

X = df[[f"f{i}" for i in range(63)]].values
y = df["label"].values

print(X.shape, y.shape)


(3871, 63) (3871,)


In [3]:
le = LabelEncoder()
y_enc = le.fit_transform(y)

num_classes = len(le.classes_)
print("Number of classes:", num_classes)


Number of classes: 57


In [4]:
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_enc, test_size=0.3, stratify=y_enc, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print(X_train.shape, X_val.shape, X_test.shape)


(2709, 63) (581, 63) (581, 63)


In [6]:
from collections import Counter

counts = Counter(y_train)

RARE_THRESHOLD = 30  # à ajuster
rare_classes = [cls for cls, c in counts.items() if c < RARE_THRESHOLD]

print("Rare classes:", [le.classes_[c] for c in rare_classes])


Rare classes: ['telvza', 'train', 'ta9ra', 'demande', 'louage', 'chabeb', 'karhba', 'thnin', 'ta3raf', 'sebt', 'assam', 'a7ad', '5adamet', 'mar7ba', 'métro', 'car', 'labes', 'bousta']


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

class KeypointDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

gan_data_idx = np.isin(y_train, rare_classes)
gan_dataset = KeypointDataset(X_train[gan_data_idx], y_train[gan_data_idx])

gan_loader = DataLoader(gan_dataset, batch_size=32, shuffle=True)

In [11]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, noise_dim, num_classes, output_dim=63):
        super().__init__()
        self.label_emb = nn.Embedding(num_classes, num_classes)

        self.net = nn.Sequential(
            nn.Linear(noise_dim + num_classes, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, z, labels):
        label_vec = self.label_emb(labels)
        x = torch.cat([z, label_vec], dim=1)
        return self.net(x)


In [12]:
class Discriminator(nn.Module):
    def __init__(self, num_classes, input_dim=63):
        super().__init__()
        self.label_emb = nn.Embedding(num_classes, num_classes)

        self.net = nn.Sequential(
            nn.Linear(input_dim + num_classes, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x, labels):
        label_vec = self.label_emb(labels)
        x = torch.cat([x, label_vec], dim=1)
        return self.net(x)


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

G = Generator(noise_dim=50, num_classes=num_classes).to(device)
D = Discriminator(num_classes=num_classes).to(device)

criterion = nn.BCELoss()
opt_G = torch.optim.Adam(G.parameters(), lr=0.0002)
opt_D = torch.optim.Adam(D.parameters(), lr=0.0002)

EPOCHS = 200


In [14]:
for epoch in range(EPOCHS):
    for real_x, labels in gan_loader:
        real_x, labels = real_x.to(device), labels.to(device)
        batch_size = real_x.size(0)

        # ========== Train D ==========
        z = torch.randn(batch_size, 50).to(device)
        fake_x = G(z, labels)

        real_loss = criterion(D(real_x, labels), torch.ones(batch_size, 1).to(device))
        fake_loss = criterion(D(fake_x.detach(), labels), torch.zeros(batch_size, 1).to(device))

        loss_D = real_loss + fake_loss
        opt_D.zero_grad()
        loss_D.backward()
        opt_D.step()

        # ========== Train G ==========
        z = torch.randn(batch_size, 50).to(device)
        fake_x = G(z, labels)
        loss_G = criterion(D(fake_x, labels), torch.ones(batch_size, 1).to(device))

        opt_G.zero_grad()
        loss_G.backward()
        opt_G.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch} | D loss: {loss_D.item():.4f} | G loss: {loss_G.item():.4f}")


Epoch 0 | D loss: 1.2708 | G loss: 0.7072
Epoch 20 | D loss: 0.8223 | G loss: 2.2517
Epoch 40 | D loss: 0.3726 | G loss: 2.0128
Epoch 60 | D loss: 0.8990 | G loss: 2.1022
Epoch 80 | D loss: 0.5825 | G loss: 2.3849
Epoch 100 | D loss: 0.8169 | G loss: 1.3309
Epoch 120 | D loss: 0.6822 | G loss: 1.6627
Epoch 140 | D loss: 1.0217 | G loss: 1.5934
Epoch 160 | D loss: 0.4959 | G loss: 2.1412
Epoch 180 | D loss: 0.7656 | G loss: 1.4080


In [15]:
def generate_samples(generator, label, n_samples):
    generator.eval()
    z = torch.randn(n_samples, 50).to(device)
    labels = torch.tensor([label] * n_samples).to(device)
    with torch.no_grad():
        samples = generator(z, labels)
    return samples.cpu().numpy()


In [16]:
X_synth = []
y_synth = []

TARGET_PER_CLASS = 50

for cls in rare_classes:
    n_to_generate = TARGET_PER_CLASS - counts[cls]
    if n_to_generate <= 0:
        continue

    samples = generate_samples(G, cls, n_to_generate)
    X_synth.append(samples)
    y_synth.extend([cls] * n_to_generate)

X_synth = np.vstack(X_synth)
y_synth = np.array(y_synth)


In [17]:
X_train_aug = np.vstack([X_train, X_synth])
y_train_aug = np.concatenate([y_train, y_synth])

print("Before GAN:", len(X_train))
print("After GAN:", len(X_train_aug))


Before GAN: 2709
After GAN: 3235


In [20]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=7,                # nb d'epochs sans amélioration
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    "best_tsl_mlp_model.h5",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [21]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(63,)),
    
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(64, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 57)             │         3,705 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,921 (81.72 KB)

 Trainable params: 20,537 (80.22 KB)

 Non-trainable params: 384 (1.50 KB)

In [22]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train_aug),
    y=y_train_aug
)

class_weights = dict(enumerate(class_weights))


In [23]:
model.fit(
    X_train_aug, y_train_aug,
    validation_data=(X_val, y_val),
    epochs=75,
    batch_size=32,
    callbacks=[early_stop, checkpoint],
    class_weight=class_weights
)


Epoch 1/75


I0000 00:00:1765766532.589492     130 service.cc:148] XLA service 0x7f54740079b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765766532.590913     130 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1765766532.977667     130 cuda_dnn.cc:529] Loaded cuDNN version 90300


 69/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0421 - loss: 4.2759 

I0000 00:00:1765766534.626645     130 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0576 - loss: 4.1412
Epoch 1: val_loss improved from inf to 3.50336, saving model to best_tsl_mlp_model.h5


...

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7480 - loss: 0.7515 - val_accuracy: 0.7952 - val_loss: 0.7251
Epoch 55/75
 92/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7405 - loss: 0.7267
Epoch 55: val_loss did not improve from 0.72508
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7411 - loss: 0.7252 - val_accuracy: 0.7831 - val_loss: 0.7528
Epoch 56/75
 92/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7522 - loss: 0.7017
Epoch 56: val_loss did not improve from 0.72508
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7509 - loss: 0.7064 - val_accuracy: 0.7952 - val_loss: 0.7271
Epoch 57/75
 92/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7413 - loss: 0.7092
Epoch 57: val_loss did not improve from 0.72508
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7419 - loss: 0.7103 - val_accuracy: 0.8158 - val_loss: 0.7333
Epoch 58/75
 90/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7358 - loss: 0.7442
Epoch 58: val_loss did not improve from 

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7372 - loss: 0.7666 - val_accuracy: 0.7831 - val_loss: 0.7189
Epoch 62/75
 92/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7378 - loss: 0.7412
Epoch 62: val_loss improved from 0.71891 to 0.70701, saving model to best_tsl_mlp_model.h5


102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7374 - loss: 0.7422 - val_accuracy: 0.7969 - val_loss: 0.7070
Epoch 63/75
 92/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7490 - loss: 0.7178
Epoch 63: val_loss did not improve from 0.70701
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7488 - loss: 0.7209 - val_accuracy: 0.8003 - val_loss: 0.7123
Epoch 64/75
 92/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7406 - loss: 0.7811
Epoch 64: val_loss improved from 0.70701 to 0.69443, saving model to best_tsl_mlp_model.h5


102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7405 - loss: 0.7783 - val_accuracy: 0.8038 - val_loss: 0.6944
Epoch 65/75
 88/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7407 - loss: 0.6917
Epoch 65: val_loss did not improve from 0.69443
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7408 - loss: 0.6987 - val_accuracy: 0.7745 - val_loss: 0.7345
Epoch 66/75
 88/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7371 - loss: 0.7186
Epoch 66: val_loss did not improve from 0.69443
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7369 - loss: 0.7230 - val_accuracy: 0.8107 - val_loss: 0.7145
Epoch 67/75
 91/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7360 - loss: 0.7034
Epoch 67: val_loss did not improve from 0.69443
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7373 - loss: 0.7032 - val_accuracy: 0.7900 - val_loss: 0.7000
Epoch 68/75
 92/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7461 - loss: 0.7637
Epoch 68: val_loss improved from 0.69443

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7466 - loss: 0.7585 - val_accuracy: 0.8038 - val_loss: 0.6796
Epoch 69/75
 90/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7441 - loss: 0.7405
Epoch 69: val_loss did not improve from 0.67957
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7438 - loss: 0.7408 - val_accuracy: 0.7952 - val_loss: 0.6961
Epoch 70/75
 91/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7594 - loss: 0.7067
Epoch 70: val_loss did not improve from 0.67957
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7590 - loss: 0.7071 - val_accuracy: 0.8003 - val_loss: 0.6925
Epoch 71/75
 88/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7598 - loss: 0.6850
Epoch 71: val_loss did not improve from 0.67957
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7604 - loss: 0.6858 - val_accuracy: 0.7917 - val_loss: 0.7109
Epoch 72/75
 91/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7391 - loss: 0.7705
Epoch 72: val_loss did not improve from 

In [27]:
import numpy as np
from sklearn.metrics import classification_report

# Charger le meilleur modèle sauvegardé
best_model = tf.keras.models.load_model("best_tsl_mlp_model.h5")

y_pred = np.argmax(best_model.predict(X_test), axis=1)

print(classification_report(
    y_test,
    y_pred,
    target_names=le.classes_
))


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
              precision    recall  f1-score   support

     3aslema       0.86      0.86      0.86         7
       3ayla       0.70      0.78      0.74         9
     5adamet       0.67      1.00      0.80         2
     5al-3am       0.95      0.64      0.77        28
        5mis       1.00      1.00      1.00         8
         5ou       0.73      0.86      0.79        28
        a7ad       0.71      0.83      0.77         6
       assam       0.71      0.83      0.77         6
     baladya       0.78      0.88      0.82        16
       banka       0.67      0.89      0.76         9
    barnamjk       0.93      1.00      0.96        13
        bent       0.79      0.46      0.58        24
         bou       0.77      0.95      0.85        21
      bousta       0.00      0.00      0.00         4
         car       0.57      0.80      0.67         5
      chabeb       0.00      0.00      0.00         6
          cv       1.00      1.00      1.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# modèle final
best_model.save("tsl_mlp_final.h5")

# encodeur des labels (déjà créé, rappel)
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# y_test et y_pred déjà calculés
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(18, 18))
sns.heatmap(
    cm,
    xticklabels=le.classes_,
    yticklabels=le.classes_,
    cmap="Blues",
    square=True,
    cbar=True
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix — Tunisian Sign Language (MLP)")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [70]:
confusions = []

for i in range(len(le.classes_)):
    for j in range(len(le.classes_)):
        if i != j and cm[i, j] > 0:
            confusions.append((
                le.classes_[i],
                le.classes_[j],
                cm[i, j]
            ))

# Trier par nombre d'erreurs
confusions = sorted(confusions, key=lambda x: x[2], reverse=True)

confusions[:15]


[('bent', 'eben', 5),
 ('o5t', '5ou', 5),
 ('o5t', 'mar2a', 5),
 ('5al-3am', 'sbitar', 4),
 ('5al-3am', 'thleth', 4),
 ('jadda', 'jad', 4),
 ('se7a', 'car', 4),
 ('5ou', 'o5t', 3),
 ('eben', 'bent', 3),
 ('mostawsaf', 'siye7a', 3),
 ('se7a', 'chabeb', 3),
 ('3ayla', 'radio', 2),
 ('5al-3am', 'baladya', 2),
 ('bent', 'mar2a', 2),
 ('bou', 'sebt', 2)]